In [1]:
###Importation des packages

import numpy as np
import time

In [2]:
def generation(n, alpha=2, beta=0, gamma=1, delta=0):
    # Initialize samples array with zeros
    sample = np.zeros(n)
    
    # Constants that do not depend on the sample index and thus can be computed once
    if alpha != 1:
        S_alpha_beta = (1 + beta ** 2 * np.tan(np.pi * alpha / 2) ** 2) ** (1 / (2 * alpha))
        B_alpha_beta = (1 / alpha) * np.arctan(beta * np.tan(np.pi * alpha / 2))

    for i in range(n):
        U = np.random.uniform(-np.pi/2, np.pi/2)
        W = -np.log(1 - np.random.uniform(0,1))
        
        # Handle the case alpha = 1 separately
        if alpha != 1:
            part1 = np.sin(alpha * (U + B_alpha_beta)) / (np.cos(U) ** (1 / alpha))
            part2 = (np.cos(U - alpha * (U + B_alpha_beta)) / W) ** ((1 - alpha) / alpha)
            sample[i] = S_alpha_beta * part1 * part2
        else:
            sample[i] = (2 / np.pi) * ((np.pi / 2 + beta * U) * np.tan(U) - beta * np.log((np.pi / 2 * W * np.cos(U))/(np.pi+beta*U)))

    # Apply scaling and location shifting
    sample = gamma * sample + delta
    return sample

In [32]:
def char_fun_zolo(t, nu, etha, tau):
    """characteristic function to use from S2 to S5

    Parameters
    ----------------------------
    t : float
    the "instant" we want to check

    nu : float
    parameter of the alpha-stable distribution, >= 1/4.

    etha :float
    parameter of the alpha-stable distribution, np.sign(etha) <= min(1, 2*np.sqrt(nu)-1)

    tau : float
    parameter of the alpha-stable distribution, absolute value < inf
    ------------------------------
    """

    y = -np.exp((nu**(-1/2))*(np.log(np.abs(t)) + tau - 1j*(np.pi/2)*etha*np.sign(t)) + np.e*((nu**(-1/2))-1))
    return y
    

In [49]:
def emp_char_fun(sample, t):
    """function to evaluate the empirical characteristic of a function at an instant t

    Parameters
    -------------------------
    sample : array-like
    sample on which we want to evaluate the characteristic function

    t : int, float
    moment to estimate/generate
    --------------------------
    """
    expo_transfo = np.exp(sample*1j*t)
    empirical = np.mean(expo_transfo)
    return empirical
    

$\theta = (\alpha, \beta, \gamma, \delta)$ 

N = 1000

$x^1,...,x^p$ ~ $\pi(x|\theta)$

$\pi_{LF}(\theta|y)$ = $\pi(\theta)\mathbb{E}_{\pi(x|\theta)}[K_{\epsilon} (y-x)]$ $\leftarrow$ we do monte carlo on this

$K_{\epsilon} (y-x)$ def by $S(y)$ ~ $N(S(x), \epsilon^2 \hat{\Sigma})$ $\leftarrow$ that would be our gaussian kernel

$\hat{\Sigma}$ is an estimate of $Cov(S(x)\hat{\theta})$

$\Lambda$ = diag(0.25, 0.25, 1, 1)

Mutation kernel : $M_t(\theta_t) = \sum_{i=1}^{N} W_{t-1}^{(i)}(\theta_{t-1}^{(i)}) \phi(\theta_t ; \theta_{t-1}^{(i)})$

Gaussian kernel : $L(x,t) = \sum_{n = -M}^{M}f(x-n)G(n,t)$

with G(n,t) = $\frac{1}{\sqrt{2\pi t}}e^{-\frac{n^2}{2t}}$


As t increases, $\epsilon_t$ decreases

$\hat{c_t}$ is the 90th quantile of the weights

In [3]:
{}

{}

# Seed : 256652

In [4]:
np.random.seed(256652)

#creation des prior des parametres cf section 3.1

prior_alpha = np.random.uniform(1.1, 2., size=1000)
prior_beta = np.random.uniform(-1., 1, size=1000)
prior_gamma = np.random.uniform(0., 300., size=1000)
prior_delta = np.random.uniform(-300., 300., size=1000)

#scale parameters

#epsilon_t

mille = np.linspace(start=100, stop=1000, endpoint=True, num=10)[::-1]
cent = np.linspace(start=10, stop=100, endpoint=False, num=90)[::-1]
dix = np.linspace(start=5, stop=10, endpoint=False, num=10)[::-1]
cinq = np.linspace(start=3, stop=5, endpoint=False, num=40)[::-1]
trois = np.linspace(start=0, stop=3, endpoint=False, num=300)[::-1]

scale_param = np.concatenate((mille, cent, dix, cinq, trois))


N = 1000
varcov_lambda = np.array([0.25,0.,0.,0.,0.,0.25, 0., 0.,0.,0.,1.,0.,0.,0.,0.,1]).reshape([4,4])

In [5]:
def gaussian_ker(u=0, y=0):
    """gaussian kernel for weights
    
    Parameters
    -------------------
    y : float
    the point we have, the output

    u : float
    the point from which we want to calculate a weight
    ----------
    """

    w = (1/np.sqrt(2*np.pi))*np.exp(-(u-y)**2/2)
    return w

In [6]:
def mc_culloch_q(data, gamma=1):
    """fonction pour calculer les quantiles de mc_culloch utilises dans le papier

    Parameters
    --------------
    data : array
    les donnes pour lesquelles on veut calculer les quantiles

    gamma : float
    valeur 1 par defaut, d'apres le papier, on utilise le gamma utilise pour faire les simulations
    ----------------
    """
    data_sorted = np.sort(data)
    useful_quantiles = np.quantile(a=data_sorted, q=[0.95, 0.75, 0.5, 0.25, 0.05])
    q_95 = useful_quantiles[0]
    q_75 = useful_quantiles[1]
    q_50 = useful_quantiles[2]
    q_25 = useful_quantiles[3]
    q_05 = useful_quantiles[4]
    alpha_hat = (q_95-q_05)/(q_75-q_25)
    beta_hat = (q_95+q_05+2*q_50)/(q_95-q_05)
    gamma_hat = (q_75-q_25)/gamma
    #on a le gamma au denominateur parce que c'est comme ca qu on a genere nos donnees
    #d'apres le papier, ca va de prendre le gamma qu'on a utililse pour generer
    delta_hat = np.mean(data_simulation)
    S_1 = np.transpose(np.array((alpha_hat, beta_hat, gamma_hat, delta_hat)))
    return S_1

In [42]:
def zolotarev_transfo(sample, xi):
    """function to use for the estimation based on the zolotarev transformation

    Parameters
    --------------------------
    Sample : array-like
    Sample to do the transformation on

    xi : int, float
    The constant used in the transformation
    --------------------------
    """
    if xi<=0 or xi>1/2 :
        raise ValueError('Xi must be between 0 and 1/2')
    taille = len(sample)
    Z = []
    for i in range(int(taille/3)):
        transfo = sample[3*i-2] - xi*sample[3*i-1] - (1 - xi)*sample[3*i]
        Z.append(transfo)
    V = []
    U = []
    for i in range(len(Z)):
        V.append(np.log(np.abs(Z[i])))
        U.append(np.sign(sample[i]))
    V = np.array(V)
    U = np.array(U)
    S_U_squared = (np.std(U))**2
    S_V_squared = (np.std(V))**2
    nu_tilde = (6/(np.pi)**2)*S_V_squared - (3/2)*S_U_squared + 1
    etha_hat = np.mean(U)
    tau_hat = np.mean(V)
    nu_hat = 0
    if nu_tilde > ((1+np.abs(etha_hat))**2)/4:
        nu_hat = nu_tilde
    else:
        nu_hat = ((1+np.abs(etha_hat))**2)/4
    delta_hat = np.mean(sample)
    S_2 = np.array((nu_hat, etha_hat, tau_hat, delta_hat))
    return S_2

In [47]:
test = generation(n=100000, alpha=1, beta=0, gamma=1, delta=0)
S_hat = zolotarev_transfo(test, xi=0.15)
print(S_hat)

[0.9977161  0.00909009 0.70168658 0.00157618]


In [48]:
#ca converge tres tres lentement

In [7]:
%%capture
#data generation and definition of moments

data_simulation = generation(n=1000, alpha=2, beta=0.5, gamma=3, delta=4)

In [8]:
%%capture
#sort data to be sure the quantiles are well computed bc you never know
data_simulation = np.sort(data_simulation)

In [9]:
useful_quantiles = np.quantile(a=data_simulation, q=[0.95, 0.75, 0.5, 0.25, 0.05])

In [10]:
useful_quantiles

array([10.84727521,  6.72913814,  3.91121601,  1.43537277, -2.13720908])

In [11]:
q_95 = useful_quantiles[0]
q_75 = useful_quantiles[1]
q_50 = useful_quantiles[2]
q_25 = useful_quantiles[3]
q_05 = useful_quantiles[4]

### Mc Culloch's quantiles

In [12]:
alpha_hat = (q_95-q_05)/(q_75-q_25)
beta_hat = (q_95+q_05+2*q_50)/(q_95-q_05)
gamma_hat = (q_75-q_25)/3
#on a le 3 au denominateur parce que c'est comme ca qu on a genere nos donnees
#d'apres le papier, ca va de prendre le gamma qu'on a utililse pour generer
delta_hat = np.mean(data_simulation)

In [13]:
S_1 = np.transpose(np.array((alpha_hat, beta_hat, gamma_hat, delta_hat)))

In [14]:
S_1

array([2.45278802, 1.27325027, 1.76458846, 4.10787777])

In [15]:
delta_hat

4.107877774019469

Test runned with one million simulations, the estimators seemed to be biased.

def prior_lf_mc():
    "estimation monte carlo de la vraisemblance
    

def smc_prc_abc(N=1000, epsilon_t):
    """Algorithme ABC de l'appendix A de l'article.


    Parameters
    ---------------------------
    N : int
    Nombre d'echantillons crees pour les parametres

    epsilon_t : array
    Les marges d'acceptation pour l'algorithme
    """
    #les prior sont des uniformes cf partie 3.1
    prior_alpha = np.random.uniform(1.1, 2., size=N)
    prior_beta = np.random.uniform(-1., 1, size=N)
    prior_gamma = np.random.uniform(0., 300., size=N)
    prior_delta = np.random.uniform(-300., 300., size=N)
    prior_gen = np.vstack((prior_alpha,prior_beta,prior_delta, prior_gamma))
    
    #the weights
    

    

    

In [17]:
a = np.array((1,2,3,4,5))

In [18]:
a

array([1, 2, 3, 4, 5])

In [19]:
b = np.array((5,4,3,2,1))

In [20]:
a/b

array([0.2, 0.5, 1. , 2. , 5. ])

In [21]:
c = np.array((5,1,3,2,4))

In [22]:
test = np.vstack((a,b,c))

In [23]:
test

array([[1, 2, 3, 4, 5],
       [5, 4, 3, 2, 1],
       [5, 1, 3, 2, 4]])

In [24]:
np.linalg.norm(a - b)

6.324555320336759

In [ ]:
test[:,0]

In [50]:
def accept_reject_abc(N=1000, epsilon_t=0.5, alpha=2, beta=0, gamma=1, delta=0):
    """algorithme d'acceptation rejet 'basique', vu en cours

    Parameters
    ---------------------------------
    N : int
    nombre de simulations faites

    epsilon_t : array
    scale parameters, marge d'acceptation

    alpha, beta, gamma, delta :
    parametres pour les simulations
    -------------------------------
    """
    data = generation(n=N, alpha=alpha, beta=beta, gamma=gamma, delta=delta)
    #priors
    prior_alpha = np.random.uniform(1.1, 2., size=5*N)
    prior_beta = np.random.uniform(-1., 1, size=5*N)
    prior_gamma = np.random.uniform(0., 300., size=5*N)
    prior_delta = np.random.uniform(-300., 300., size=5*N)
    prior_gen = np.vstack((prior_alpha,prior_beta,prior_delta, prior_gamma))
    true_data_estim = zolotarev_transfo(sample=data, xi=0.15)
    bon_param = np.array(())
    alpha_test = 0
    beta_test = 0
    gamma_test = 0
    delta_test = 0
    for i in range(5*N):
        proposed_data = generation(n=N, alpha=prior_gen[:,i][0],
                                   beta=prior_gen[:,i][1],
                                   delta=prior_gen[:,i][2],
                                   gamma=prior_gen[:,i][3])
        alpha_test = prior_gen[:,i][0]
        beta_test = prior_gen[:,i][1]
        gamma_test = prior_gen[:,i][2]
        delta_test = prior_gen[:,i][3]
        estimated = mc_culloch_q(proposed_data, gamma=prior_gen[:,i][3])
        weights = gaussian_ker(estimated, true_data_estim)
        if weights[0] and weights[1] and weights[2] and weights[3] < epsilon_t:
            return alpha_test, beta_test, gamma_test, delta_test
        else :
            continue

In [51]:
test_1 = accept_reject_abc(N=1000, epsilon_t=0.3)

In [52]:
start = time.time()
test_1
end = time.time()
print(test_1, end - start)

(1.4033620062901593, 0.813262090631643, -247.9279315952059, 257.3977994913487) 0.00011873245239257812


In [ ]:
#avec une si petite distance ca ne marchait pas

In [56]:
start = time.time()
liste_parametres = []
for i in range(len(scale_param)):
    pouet = accept_reject_abc(N=1000, epsilon_t=scale_param[-i], alpha=1, beta=0, gamma=1, delta=0)
    intermediaire = [scale_param[-i], pouet]
    liste_parametres.append(intermediaire)

end = time.time()

print(end - start)

139.70867395401


In [57]:
#must change the parameters, N must be 200, too long otherwise

In [58]:
liste_parametres[:50]

[[1000.0,
  (1.988826864246049,
   0.6132475309237018,
   -93.22791502564576,
   58.447200164146466)],
 [0.0, None],
 [0.01,
  (1.4684996341178636,
   -0.7412219101962036,
   122.42222867419628,
   126.14813594206862)],
 [0.02,
  (1.870604107440861,
   0.6497604115236333,
   154.51824405428022,
   24.616059984069206)],
 [0.03,
  (1.8340380665196174,
   0.37862484027523724,
   -8.15669623463316,
   14.570269832725224)],
 [0.04,
  (1.74582468489793,
   -0.5107989910414292,
   151.58377154167664,
   153.52932738070328)],
 [0.05, None],
 [0.06,
  (1.2024700043167522,
   -0.6867047550359855,
   -252.35684172021462,
   208.80614518427595)],
 [0.07,
  (1.5616105928333066,
   0.9968896016396838,
   -198.05101519280288,
   182.10408713751565)],
 [0.08,
  (1.2440327687808899,
   -0.43588127994348236,
   -77.16277475752912,
   139.38860924184374)],
 [0.09,
  (1.3012346253032412,
   -0.6427528632362818,
   287.37851302440333,
   36.20249918783189)],
 [0.1,
  (1.8703490888709802,
   0.5968283112536

In [ ]:
#not working very well, new distance needed

In [ ]:
#Works probably, pb : wrong use of mc culloch quantiles, need to invert --> pb , how